# Submitting to Quantinuum Emulators via pytket

This notebook contains examples for running quantum circuits on Quantinuum's emulators.

We can use an emulator to get an idea of what a quantum device might output for our quantum circuit. This enables circuit debugging and optimization before running on a physical machine.

See the *Quantinuum Systems User Guide* on the Quantinuum User Portal for detailed information on each of the systems available and workflow information including job submission, queueing, and data retention.

* [Basic Example](#example)
* [Without Noise](#no-noise)
* [Stabilizer Emulator](#stabilizer)

## Basic Example <a class="anchor" id="example"></a>

### Circuit Preparation

Create your circuit via the pytket python library. For details on getting started with `pytket`, see pytket's [Getting Started](https://cqcl.github.io/tket/pytket/api/getting_started.html) page.

In [2]:
from pytket.circuit import Circuit, fresh_symbol
from pytket.circuit.display import render_circuit_jupyter

# Set up Bell Test
circuit = Circuit(2, name="Bell Test")
circuit.H(0)
circuit.CX(0, 1)
circuit.measure_all()

render_circuit_jupyter(circuit)

### Select Device

*Available emulators:*
- `H1-1E`, `H1-2E`: Device-specific Emulators of H1-1 and H1-2

Login to the Quantinuum API using your credentials and check the device status.

In [3]:
from pytket.extensions.quantinuum import QuantinuumBackend

QuantinuumBackend.login()

Enter your Quantinuum email: megan.l.kohagen@quantinuum.com
Enter your Quantinuum password: ········


In [5]:
machine = 'H1-2E'

backend = QuantinuumBackend(device_name=machine)

print(machine, "status:", backend.device_state(device_name=machine))

H1-2E status: online


### Circuit Compilation

`pytket` includes many features for optimizing circuits. This includes reducing the number of gates where possible and resynthesizing circuits for a quantum computer's native gate set. See the `pytket` [User Manual](https://cqcl.github.io/pytket/manual/index.html) for more information on all the options that are available.

Here the circuit is compiled with `get_compiled_circuit`, which includes optimizing the gates and resynthesizing the circuit to Quantinuum's native gate set. The `optimisation_level` sets the level of optimisation to perform during compilation, check pytket documentation for more details.

In [6]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=1)

render_circuit_jupyter(compiled_circuit)

### Run the Circuit

Now the circuit can be run on an emulator.

In [21]:
n_shots = 100
handle = backend.process_circuit(compiled_circuit, 
                                 n_shots=n_shots)
print(handle)

('25385b6bf52f4a1dbb578f811f9d39d4', 'null')


Check the status of the job.

In [22]:
status = backend.circuit_status(handle)
print(status)

CircuitStatus(status=<StatusEnum.QUEUED: 'Circuit is queued.'>, message="{'name': 'Bell Test', 'submit-date': '2022-04-04T19:24:06.557667', 'result-date': None, 'queue-position': None, 'cost': '5.66', 'error': None}", error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


### Retrieve Results

Once a job's status returns completed, results can be returned using the `get_result` function.

In [23]:
result = backend.get_result(handle)

result

BackendResult(q_bits={},c_bits={c[1]: 0, c[0]: 1},counts=None,shots=[[192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [128]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [128]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [192]],state=None,unitary=None,density_matrix=None)

### Save Results

It is recommended that users save job results as soon as jobs are completed due to the Quantinuum data retention policy.

In [ ]:
import json

with open('pytket_emulator_example.json', 'w') as file:
    json.dump(result.to_dict(), file)

### Analyze Results

The result output is just like that of a quantum device. 

The simulation by default runs with noise; therefore, it won't give back all `11`'s.

In [24]:
result = backend.get_result(handle)
print(result.get_distribution())

{(0, 0): 0.48, (0, 1): 0.02, (1, 1): 0.5}


In [25]:
print(result.get_counts())

Counter({(1, 1): 50, (0, 0): 48, (0, 1): 2})


## Without Noise <a class="anchor" id="no-noise"></a>

The Quantinuum emulators may be run with or without the physical device noise model. We can set `noisy_simulation=False` to do this.

In [7]:
n_shots = 100
no_error_model_handle = backend.process_circuit(compiled_circuit, 
                                                 n_shots=n_shots,
                                                noisy_simulation=False)
print(no_error_model_handle)

('b6d263313ba74de2a497cfc48fcc3c56', 'null')


In [8]:
no_error_model_status = backend.circuit_status(no_error_model_handle)
print(no_error_model_handle)

('b6d263313ba74de2a497cfc48fcc3c56', 'null')


In [9]:
no_error_model_result = backend.get_result(no_error_model_handle)

no_error_model_result

BackendResult(q_bits={},c_bits={c[1]: 0, c[0]: 1},counts=None,shots=[[192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [  0]],state=None,unitary=None,density_matrix=None)

In [ ]:
with open('pytket_emulator_no_error_model_example.json', 'w') as file:
    json.dump(result.to_dict(), file)

In [11]:
no_error_model_result = backend.get_result(no_error_model_handle)
print(no_error_model_result.get_distribution())

print(no_error_model_result.get_counts())

{(0, 0): 0.46, (1, 1): 0.54}
Counter({(1, 1): 54, (0, 0): 46})


## Stabilizer Emulator <a class="anchor" id="stabilizer"></a>

By default, emulations are run using a state-vector emulator, which simulates any quantum operation. However, if the quantum operations are all Clifford gates, it can be faster for complex circuits to use the `stabilizer` emulator, which can be requested as follows:

In [21]:
n_shots = 100
stabilizer_handle = backend.process_circuit(compiled_circuit, 
                                            n_shots=n_shots)
print(stabilizer_handle)

('25385b6bf52f4a1dbb578f811f9d39d4', 'null')


In [22]:
stabilizer_status = backend.circuit_status(stabilizer_handle)
print(stabilizer_handle)

CircuitStatus(status=<StatusEnum.QUEUED: 'Circuit is queued.'>, message="{'name': 'Bell Test', 'submit-date': '2022-04-04T19:24:06.557667', 'result-date': None, 'queue-position': None, 'cost': '5.66', 'error': None}", error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


In [23]:
stabilizer_result = backend.get_result(stabilizer_handle)

stabilizer_result

BackendResult(q_bits={},c_bits={c[1]: 0, c[0]: 1},counts=None,shots=[[192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [128]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [  0]
 [128]
 [192]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [192]
 [192]
 [192]
 [  0]
 [192]
 [192]
 [  0]
 [  0]
 [  0]
 [192]
 [  0]
 [192]
 [  0]
 [192]
 [192]],state=None,unitary=None,density_matrix=None)

In [ ]:
with open('pytket_emulator_stabilizer_example.json', 'w') as file:
    json.dump(result.to_dict(), file)

In [8]:
stabilizer_result = backend.get_result(stabilizer_handle)
print(stabilizer_result.get_distribution())

print(stabilizer_result.get_counts())

number of '11's: 1000
number of results: 1000


<div align="center"> &copy; 2022 by Quantinuum. All Rights Reserved. </div>